# Installing Spotipy

In order to use the Spotify API (SpotiPy) we will have to create an account in Spotify and follow these steps. Once we have done it we will start initializing the API and look at the search method for which we can introduce a "query" q, in this example we will try it with Lady Gaga:

In [1]:
##!conda install -c conda-forge spotipy

## Loading credentials from another config file

In [2]:
import config

## Starting with Spotify API

In [3]:
import spotipy
import json
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [5]:
# get playlists
results = sp.user_playlists('spotify', limit= 20)

In [6]:
results.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [7]:
results['items'][0].keys()

dict_keys(['collaborative', 'description', 'external_urls', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])

In [8]:
results['items'][0]

{'collaborative': False,
 'description': 'Miley Cyrus is on top of the Hottest 50!',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DXcBWIGoYBM5M'},
 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXcBWIGoYBM5M',
 'id': '37i9dQZF1DXcBWIGoYBM5M',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab67706f00000003a03167767495b25d14b9f2ea',
   'width': None}],
 'name': "Today's Top Hits",
 'owner': {'display_name': 'Spotify',
  'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MTY3NjM5NTgyNywwMDAwMDAwMDY1MzkwMTYwZTY2ZGM1N2YxZTM1YThjOWVlZmZiZTU3',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DXcBWIGoYBM5M/tracks',
  'total': 50},
 'type': 'playlist',
 'uri': 'spotify:playlist:37i9dQZF1DXcBWIGoYBM5M'}

In [9]:
# print playlist IDs
playlists_ids = []
for playlist in results['items']:
    playlist_ids = playlist['id']
    playlists_ids.append(playlist['id'])
playlists_ids

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX1lVhptIYRda',
 '37i9dQZF1DX10zKzsJ2jva',
 '37i9dQZF1DX4JAvHpjipBk',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DX4SBhb3fqCJd',
 '37i9dQZF1DWXRqgorJj26U',
 '37i9dQZF1DX4dyzvuaRJ0n',
 '37i9dQZF1DXcF6B6QPhFDv',
 '37i9dQZF1DWXJfnUiYjUKT',
 '37i9dQZF1DXcRXFNfZr7Tp',
 '37i9dQZF1DX4o1oenSJRJd',
 '37i9dQZF1DXbTxeAdrVG2l',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DWTJ7xPn4vNaz',
 '37i9dQZF1DXaKIA8E7WcJj',
 '37i9dQZF1DWSV3Tk4GO2fq',
 '37i9dQZF1DWTwnEm1IYyoj',
 '37i9dQZF1DX2A29LI7xHn1']

In [10]:
# get the all information of playlist
# username = 'Spotify'
# playlist_id = '37i9dQZF1DWX83CujKHHOn'# get from url of the playlist

# playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DWX83CujKHHOn",market="GB")
# print(playlist)

In [11]:
username = 'Spotify'

In [12]:
# build a function to extract the tracks from the playlist.
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [13]:
list_of_audio_features=[]
for playlist_id in playlists_ids: 
    tracks = get_playlist_tracks("Spotify", playlist_id)
    
    for item in range(0,len(tracks)):
    #print (tracks[item]["track"]["id"])
        if tracks[item] is not None and tracks[item]["track"] is not None:
            audio_features = sp.audio_features(tracks[item]["track"]["id"])
            if audio_features is not None and len(audio_features) > 0:
                list_of_audio_features.append(audio_features[0])
#                     list_of_audio_features.append(sp.audio_features(tracks[item]["track"]["id"])[0])
        

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [ ]:
df= pd.DataFrame(list_of_audio_features)    
df= df[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

df

In [ ]:
len(list_of_audio_features)

### Add artists' names

In [ ]:
def get_tracks_artists(username, playlist_id):
    tracks = get_playlist_tracks(username, playlist_id)
    if tracks is not None:
        artists = []
        for item in range(0, len(tracks)):
            artists.append(tracks[item]['track']['artists'][0]['name'])
        return artists
    else:
        return []

In [ ]:
artist_list = []

for playlist_id in playlists_ids:
    artist_list.extend(get_tracks_artists('Spotify', playlist_id))

In [ ]:
len(artist_list)

In [ ]:
df['Artists'] = artist_list

In [ ]:
df

### Add albums' year

In [ ]:
tracks[0]['track']['album']['release_date']

In [ ]:
def get_tracks_year(username, playlist_id):
    tracks = get_playlist_tracks(username, playlist_id)
    if tracks is not None:
        years = []
        for item in range(0, len(tracks)):
            years.append(tracks[0]['track']['album']['release_date'])
        return years
    else:
        return []

In [ ]:
years_list = []

for playlist_id in playlists_ids:
    years_list.extend(get_tracks_year('Spotify', playlist_id))

In [ ]:
df['Years'] = years_list

In [ ]:
df

### Add albums' names

In [ ]:
# album's names
tracks[0]['track']['album']['name']

In [ ]:
def get_tracks_albumname(username, playlist_id):
    tracks =  get_playlist_tracks(username, playlist_id)
    if tracks is not None:
        album_name = []
        for item in range(0, len(tracks)):
            album_name.append(tracks[item]['track']['album']['name'])
        return album_name
    else:
        return []

In [ ]:
album_name_list = []

for playlist_id in playlists_ids:
    album_name_list.extend(get_tracks_albumname('Spotify', playlist_id))

In [ ]:
df['Album name']= album_name_list

### Add songs' names

In [ ]:
# songs' names
tracks[0]['track']['name']

In [ ]:
def get_tracks_songname(username, playlist_id):
    tracks =  get_playlist_tracks(username, playlist_id)
    if tracks is not None:
        song_name = []
        for item in range(0, len(tracks)):
            song_name.append(tracks[item]['track']['name'])
        return song_name
    else:
        return []

In [ ]:
song_name_list = []

for playlist_id in playlists_ids:
    song_name_list.extend(get_tracks_songname('Spotify', playlist_id))

In [ ]:
df['Song name' ] =song_name_list 

### Add popularity

In [ ]:
# example: get fist song's popularity
tracks[0]['track']['popularity']

In [ ]:
# extract all songs' popularity
def get_tracks_popularity(username, playlist_id):
    tracks = get_playlist_tracks(username, playlist_id)
    if tracks is not None:
        popularity = []
        for item in range(0, len(tracks)):
            popularity.append(tracks[item]['track']['popularity'])
        return popularity
    else:
        return []

In [ ]:
popularity_list = []

for playlist_id in playlists_ids:
    popularity_list.extend(get_tracks_popularity('Spotify', playlist_id))

In [ ]:
df['Popularity']= popularity_list

In [ ]:
df

In [ ]:
song_list.to_csv('Spotify.csv', index = False)